# For Windows
1. Please download the latest SASview v5 package from
   https://github.com/SasView/sasview/tags
2. Install SASview
3. copy the *sasmodels* directory here (it will be found in the SASview install directory)

## In case of error
<pre style="background: #fdd">
    CalledProcessError: Command '['gcc', ...
</pre>
Please start the previously installed SASview at least once and create a plot with the desired model function. This will build the selected model from C-sources into a python library in the directory <br />
`C:\Users\{user}\.sasmodels\compiled_models\sas32_{model}.so`

If it still fails, it might be caused by OpenCL being installed while the *pyopencl* package can not be imported in jupyter notebook/lab. Try to disable OpenCL entirely by setting `os.environ["SAS_OPENCL"] = "none"` early in the code and check that the desired SASview model was build already (see above).

# McSAS example

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import sys, os
# disable OpenCL if it causes problems
#os.environ["SAS_OPENCL"] = "none"
# set the location where the sasview-sasmodels are installed:
sasmodelsPath = os.path.join(os.getcwd(), "sasmodels")
sys.path.append(sasmodelsPath)
from mcsas3 import mcmodel, mcopt, mccore

%matplotlib inline
import matplotlib.pyplot as plt
from pandas import read_csv

In [ ]:
# import data
qs = read_csv("quickstartdemo1.csv", delimiter = ';',
              header = None, names = ["Q", "I", "ISigma"])
# Q needs to be in a list as in [Q] or [Qx,Qy]
# measured intensity I (same shape as Q) and uncertainties ISigma
measData = dict(Q = [qs.Q.values], I = qs.I.values,
                ISigma = qs.ISigma.values)

# initialize optimization settings
opt = mcopt.McOpt(maxIter = 1e5, convCrit = 1)
# create a model
mod = mcmodel.McModel(modelName = "sphere", nContrib = 300,
                      fitParameterLimits = {"radius": (3, 315)}, 
                      staticParameters = {"background": 0,
                                          "sld" : 1e+6,
                                          "sld_solvent" : 0, })
mc = mccore.McCore(measData, model = mod, opt = opt)
mc.optimize()
mc._model.kernel.release()
print("Final chiSqr: {}, N accepted: {}"
      .format(mc._opt.gof, mc._opt.accepted))

# storing the results
mc._opt.repetition = 0
mc.store(filename = "results.h5")
# plotting the intensity match
plt.plot(measData["Q"][0], measData["I"], label = "Input Data")
plt.yscale("log")
plt.xscale("log")
plt.xlabel("Q (1/nm)")
plt.ylabel("I (1/m)")
scale, background = mc._opt.x0[0], mc._opt.x0[1]
plt.plot(measData["Q"][0], mc._opt.modelI*scale + background,
         label = "MC fit")
_ = plt.legend(loc = 0)